# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
F. Walter  ->  F. Walter  |  ['F. Walter']
J. Li  ->  J. Li  |  ['J. Li']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
Arxiv has 86 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.12420


extracting tarball to tmp_2412.12420...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.12601


extracting tarball to tmp_2412.12601... done.
Retrieving document from  https://arxiv.org/e-print/2412.12715


...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.13189


extracting tarball to tmp_2412.13189...

 done.


J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']


Found 90 bibliographic references in tmp_2412.13189/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.12715-b31b1b.svg)](https://arxiv.org/abs/2412.12715) | **MINDS. JWST-MIRI reveals a peculiar CO$_2$-rich chemistry in the drift-dominated disk CX Tau**  |
|| M. Vlasblom, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2024-12-18*|
|*Comments*| *23 pages, 17 figures, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            Radial drift of icy pebbles can have a large impact on the chemistry of the inner regions of protoplanetary disks. Compact dust disks ($\lesssim$50 au) are suggested to have a higher (cold) H$_2$O flux than more extended disks, likely due to efficient radial drift bringing H$_2$O-rich material to the inner disk, where it can be observed with JWST. We present JWST MIRI/MRS observations of the disk CX Tau taken as a part of the Mid-INfrared Disk Survey (MINDS) GTO program, a prime example of a drift-dominated disk. This compact disk seems peculiar: the source possesses a bright CO$_2$ feature instead of the bright H$_2$O expected based on its efficient radial drift. We aim to provide an explanation for this finding. We detect molecular emission from H$_2$O, $^{12}$CO$_2$, $^{13}$CO$_2$, C$_2$H$_2$, HCN, and OH in this disk, and even demonstrate a potential detection of CO$^{18}$O. Analysis of the $^{12}$CO$_2$ and $^{13}$CO$_2$ emission shows the former to be tracing a temperature of $\sim$450 K, whereas the $^{13}$CO$_2$ traces a significantly colder temperature ($\sim$200 K). H$_2$O is also securely detected both at shorter and longer wavelengths, tracing a similar temperature of $\sim$500-600 K as the CO$_2$ emission. We also find evidence for a colder, $\sim$200 K H$_2$O component at longer wavelengths, which is in line with this disk having strong radial drift. The cold $^{13}$CO$_2$ and H$_2$O emission indicate that radial drift of ices likely plays an important role in setting the chemistry of the inner disk of CX Tau. Potentially, the H$_2$O-rich gas has already advected onto the central star, which is now followed by an enhancement of comparatively CO$_2$-rich gas reaching the inner disk, explaining the enhancement of CO$_2$ emission in CX Tau. The comparatively weaker H$_2$O emission can be explained by the source's low accretion luminosity. (abridged)         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.12826-b31b1b.svg)](https://arxiv.org/abs/2412.12826) | **Insight into the Starburst Nature of Galaxy GN-z11 with JWST MIRI Spectroscopy**  |
|| J. Álvarez-Márquez, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-12-18*|
|*Comments*| *accepted for publication in A&A (17 pages, 6 figures, 2 tables)*|
|**Abstract**|            This paper presents a deep MIRI/JWST medium resolution spectroscopy (MRS) covering the rest-frame optical spectrum of the GN-z11 galaxy. The [OIII]5008 and H$\alpha$ emission lines are detected and spectroscopically resolved. The line profiles are well-modeled by a narrow Gaussian component with intrinsic FWHMs of 189$\pm$25 and 231$\pm$52 kms$^{-1}$, respectively. We do not find any evidence of a dominant broad H$\alpha$ emission line component tracing a Broad Line Region in a type 1 active galactic nuclei (AGN). However, a broad ($\sim$430-470 kms$^{-1}$) and weak ($<$ 20-30%) H$\alpha$ line component, tracing a minor AGN contribution in the optical, cannot be ruled out completely with the sensitivity of the present data. The physical and excitation properties of the ionized gas are consistent with a low-metallicity starburst forming stars at a rate of SFR(H$\alpha$)$=$24 $\pm$3$M_{\odot}$yr$^{-1}$. The electron temperature of the ionized gas is $T_{\mathrm{e}}$(O$^{++}$)$=$14000$\pm$2100K, while the direct-$T_{\mathrm{e}}$ gas-phase metallicity is 12+$\log$(O/H)$=$7.91$\pm$0.07 (Z=0.17$\pm$0.03Z$_{\odot}$). The optical line ratios locate GN-z11 in the starburst or AGN region but more consistent with those of local low-metallicity starbursts and high-$z$ luminous galaxies detected at redshifts similar to GN-z11. We conclude that the MRS optical spectrum of GN-z11 is consistent with that of a massive, compact, and low-metallicity starburst galaxy. Due to its high SFR and stellar mass surface densities, close to that of the densest stellar clusters, we speculate that GN-z11 could be undergoing a feedback-free, highly efficient starburst phase. Additional JWST data are needed to validate this scenario, and other recently proposed alternatives, to explain the existence of bright compact galaxies in the early Universe.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.13189-b31b1b.svg)](https://arxiv.org/abs/2412.13189) | **Binary properties of the globular cluster 47 Tuc (NGC 104). A dearth of short-period binaries**  |
|| <mark>J. Müller-Horn</mark>, et al. |
|*Appeared on*| *2024-12-18*|
|*Comments*| *Accepted for publication in Astronomy and Astrophysics, 18 pages, 20 figures*|
|**Abstract**|            Spectroscopic observations of binary stars in globular clusters are essential to shed light on the poorly constrained period, eccentricity, and mass ratio distributions and to develop an understanding of the formation of peculiar stellar objects. 47 Tuc (NGC 104) is one of the most massive Galactic globular clusters, with a large population of blue stragglers and with many predicted but as-yet elusive stellar-mass black holes. This makes it an exciting candidate for binary searches. We present a multi-epoch spectroscopic survey of 47 Tuc with the VLT/MUSE integral field spectrograph to determine radial velocity variations for 21,699 stars. We find a total binary fraction in the cluster of $(2.4\pm1.0)\%$, consistent with previous photometric estimates, and an increased binary fraction among blue straggler stars, approximately three times higher than the cluster average. We find very few binaries with periods below three days, and none with massive dark companions. A comparison with predictions from state-of-the-art models shows that the absence of such short-period binaries and of binaries with massive companions is surprising, highlighting the need to improve our understanding of stellar and dynamical evolution in binary systems.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.12420-b31b1b.svg)](https://arxiv.org/abs/2412.12420) | **Testing Colour-magnitude Pattern as A Method in the Search for Changing-Look AGNs**  |
|| L.-t. Zhu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-12-18*|
|*Comments*| *16 pages, 9 figures, 3 tables, accepted for publication in MNRAS*|
|**Abstract**|            We develop a simple method to search for changing-look (CL) active galactic nucleus (AGN) candidates, and conduct a test run. In this method, optical variations of AGNs are monitored and CL-AGNs may appear to have a pattern of being bluer when in brightening flare-like events. Applying this method, previously-classified type 2 AGNs that show the bluer-when-brighter (BWB) pattern are selected. Among more than ten thousands type 2 AGNs classified in the Sloan Digital Sky Survey (SDSS), we find 73 candidates with possibly the strongest BWB pattern. We note that 13 of them have previously been reported as CL-AGNs. We have observed nine candidates, and found that five among them showed the CL transition from type 2 to type 1. In addition, we also test extending the selection to previously-classified type 1 AGNs in the SDSS by finding sources with a possible redder-when-brighter pattern, but none of the three sources observed by us is found to show the transition from type 1 to type 2. We discuss the variation properties in both the success and failure cases, and plan to observe more candidates selected with the method. From the observational results, a detailed comparison between the CL-AGNs and none CL-AGNs will help quantitatively refine the selection criteria and in turn allow us to configure the general properties of CLAGNs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.12601-b31b1b.svg)](https://arxiv.org/abs/2412.12601) | **Minute-cadence observations on Galactic plane with Wide Field Survey Telescope (WFST): Overview, methodology and early results**  |
|| J. Lin, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-12-18*|
|*Comments*| *19 pages, 12 figures, submitted*|
|**Abstract**|            As the time-domain survey telescope of the highest survey power in the northern hemisphere currently, Wide Field Survey Telescope (WFST) is scheduled to hourly/daily/semi-weekly scan northern sky up to ~23 mag in four optical (ugri) bands. Unlike the observation cadences in the forthcoming regular survey missions, WFST performed "staring" observations toward Galactic plane in a cadence of $\approx$1 minute for a total on-source time of about 13 hours, during the commissioning and pilot observation phases. Such an observation cadence is well applied in producing densely sampling light curves and hunting for stars exhibiting fast stellar variabilities. Here we introduce the primary methodologies in detecting variability, periodicity, and stellar flares among a half million sources from the minute-cadence observations, and present the WFST g-/r-band light curves generated from periodic variable stars and flaring stars. Benefit from high photometric precisions and deep detection limits of WFST, the observations have captured several rare variable stars, such as a variable hot white dwarf (WD) and an ellipsoidal WD binary candidate. By surveying the almost unexplored parameter spaces for variables, WFST will lead to new opportunities in discovering unique variable stars in the northern sky.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.13166-b31b1b.svg)](https://arxiv.org/abs/2412.13166) | **The Red Supergiant Progenitor of the Type II Supernova 2024abfl**  |
|| J. Luo, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-12-18*|
|*Comments*| *11 pages, 6 figures, in review (submitted on Dec 9th, 2024)*|
|**Abstract**|            Linkage between core-collapse supernovae (SNe) and their progenitors is not fully understood and ongoing effort of searching and identifying the progenitors is needed. $\mathrm{SN\,2024abfl}$ is a recent Type II supernova exploded in the nearby star-bursting galaxy $\mathrm{NGC\,2146}$, which is also the host galaxy of $\mathrm{SN\,2018zd}$. From archival Hubble Space Telescope (HST) data, we have found a red source ($\mathrm{m_{F814W} \sim 25}$) near the location (angular distance $\leq 0.2"$) of $\mathrm{SN\,2024abfl}$ before its explosion. With F814W and F606W photometry, we found that the properties of this source matched a typical red supergiant (RSG) moderately reddened by interstellar dust at the distance of the host galaxy. We conclude that the $\mathrm{SN\,2024abfl}$ had an RSG progenitor with initial mass of $\mathrm{10M_{\odot}}$--$\mathrm{16\,M_{\odot}}$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.12715/./Figures/CXTau_slab_15um_ind_v8b_CS_p1_v3.png', 'tmp_2412.12715/./Figures/CXTau_slab_15um_ind_v8b_CS_p2_v3.png', 'tmp_2412.12715/./Figures/CXTau_slab_13CO2_v8b_CS_v2.png']
copying  tmp_2412.12715/./Figures/CXTau_slab_15um_ind_v8b_CS_p1_v3.png to _build/html/
copying  tmp_2412.12715/./Figures/CXTau_slab_15um_ind_v8b_CS_p2_v3.png to _build/html/
copying  tmp_2412.12715/./Figures/CXTau_slab_13CO2_v8b_CS_v2.png to _build/html/
exported in  _build/html/2412.12715.md
    + _build/html/tmp_2412.12715/./Figures/CXTau_slab_15um_ind_v8b_CS_p1_v3.png
    + _build/html/tmp_2412.12715/./Figures/CXTau_slab_15um_ind_v8b_CS_p2_v3.png
    + _build/html/tmp_2412.12715/./Figures/CXTau_slab_13CO2_v8b_CS_v2.png
found figures ['tmp_2412.12826/./Emission_line_plot_newcal.png', 'tmp_2412.12826/./Emission_line_fit_plot_1comp_newcal.png', 'tmp_2412.12826/./M_R_newcal.png']
copying  tmp_2412.12826/./Emission_line_plot_newcal.png to _build/html/
copying  tmp_2412.12826/./Emission_l

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mv}[1]{\textcolor{Thistle}{MV: #1}}$</div>



<div id="title">

# MINDS. JWST-MIRI reveals a peculiar $\ce{CO2}$-rich chemistry in the drift-dominated disk CX Tau

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.12715-b31b1b.svg)](https://arxiv.org/abs/2412.12715)<mark>Appeared on: 2024-12-18</mark> -  _23 pages, 17 figures, accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

M. Vlasblom, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** Radial drift of icy pebbles can have a large impact on the chemistry of the inner regions of protoplanetary disks, where most terrestrial planets are thought to form. Disks with compact mm dust emission ( $\lesssim$ 50 au) are suggested to have a higher $\ce{H2O}$ flux than more extended disks, as well as show excess cold $\ce{H2O}$ emission, likely due to efficient radial drift bringing $\ce{H2O}$ -rich material to the inner disk, where it can be observed with IR facilities such as the _James Webb_ Space Telescope (JWST). We present JWST MIRI/MRS observations of the disk around the low-mass T Tauri star CX Tau (M2.5, 0.37 M $_\odot$ ) taken as a part of the Mid-INfrared Disk Survey (MINDS) GTO program, a prime example of a drift-dominated disk based on ALMA data. In the context of compact disks, this disk seems peculiar: the source possesses a bright $\ce{CO2}$ feature instead of the bright $\ce{H2O}$ that could perhaps be expected based on the efficient radial drift. We aim to provide an explanation for this finding in the context of radial drift of ices and the disk's physical structure. We model the molecular features in the spectrum using local thermodynamic equilibrium (LTE) 0D slab models, which allows us to obtain estimates of the temperature, column density and emitting area of the emission. We detect molecular emission from $\ce{H2O, ^12CO2, ^13CO2, C2H2, HCN}$ , and OH in this disk, and even demonstrate a potential detection of $\ce{CO^18O}$ emission. Analysis of the $\ce{^12CO2}$ and $\ce{^13CO2}$ emission shows the former to be optically thick and tracing a temperature of $\sim$ ${450}$ K at an (equivalent) emitting radius of $\sim$ 0.05 au. The optically thinner isotopologue traces significantly colder temperatures ( $\sim$ ${200}$ K) and a larger emitting area. Both the ro-vibrational bands ${of \ce{H2O}}$ at shorter wavelengths and ${its}$ pure rotational bands at longer wavelengths are securely detected. Both sets of lines are optically thick, tracing ${a similar temperature of $\sim$500-600 K and emitting area as the \ce{CO2} emission}$ . We also find evidence for an even colder, $\sim$ ${200}$ K $\ce{H2O}$ component at longer wavelengths, which is in line with this disk having strong radial drift. We also ${find evidence of}$ highly excited rotational OH ${emission}$ at 9-11 $\mu$ m, known as `prompt emission', caused by $\ce{H2O}$ photodissociation. Additionally, we firmly detect 4 pure rotational lines of $\ce{H2}$ , ${which show evidence of extended emission}$ . Finally, we also detect several H recombination lines and the [ Ne II ] line. The cold temperatures found for both the $\ce{^13CO2}$ and $\ce{H2O}$ emission at longer wavelengths indicate that radial drift of ices likely plays an important role in setting the chemistry of ${the inner disk of CX Tau}$ . Potentially, the $\ce{H2O}$ -rich gas has already advected onto the central star, which is now followed by an enhancement of comparatively $\ce{CO2}$ -rich gas reaching the inner disk, explaining the enhancement of $\ce{CO2}$ emission in CX Tau. ${The comparatively weaker \ce{H2O} emission can be explained by the source's low accretion luminosity. Alternatively, the presence of a small, inner cavity with a size of roughly 2 au in radius, outside the \ce{H2O} iceline, could explain the bright \ce{CO2} emission.}$ Higher angular resolution ALMA observations are needed to test this.

</div>

<div id="div_fig1">

<img src="tmp_2412.12715/./Figures/CXTau_slab_15um_ind_v8b_CS_p1_v3.png" alt="Fig10" width="100%"/>

**Figure 10. -** A zoom-in of the {13.5-15.5}$\mu$m region of the CX Tau spectrum (black), where all individual slab model fits are shown. Horizontal bars indicate {which spectral windows were used} for the $\chi^2$ fit, and each panel indicates which other models were removed from the spectrum before the fit was performed, to avoid contamination from other molecules. {The final panel shows the residuals after all fits are subtracted, with the shaded region indicating that most residuals fall below 3$\sigma$. We note the difference in scale on the y axis between the top panel and the rest.} (*fig:slab_15um_ind_p1*)

</div>
<div id="div_fig2">

<img src="tmp_2412.12715/./Figures/CXTau_slab_15um_ind_v8b_CS_p2_v3.png" alt="Fig11" width="100%"/>

**Figure 11. -** A zoom-in of the {15.5-17.5}$\mu$m region of the CX Tau spectrum (black), where all individual slab model fits are shown. Horizontal bars indicate {which spectral windows were used} for the $\chi^2$ fit, and each panel indicates which other models were removed from the spectrum before the fit was performed, to avoid contamination from other molecules. {The final panel shows the residuals after all fits are subtracted, with the shaded region indicating that most residuals fall below 3$\sigma$. {An artifact around 16.15 $\mu$m has been masked in all panels.} We {also} note the difference in scale on the y axis between the top panel and the rest.} (*fig:slab_15um_ind_p2*)

</div>
<div id="div_fig3">

<img src="tmp_2412.12715/./Figures/CXTau_slab_13CO2_v8b_CS_v2.png" alt="Fig2" width="100%"/>

**Figure 2. -** A comparison of the \ce{^12CO2}(green) and \ce{^13CO2}(purple) $Q$ branch shapes. Top: {a zoom-in of the \ce{^12CO2} $Q$ branch with the best-fit slab model plotted in the green shaded region. A model {with} the derived \ce{^13CO2} parameters is plotted in purple (see text).} Middle: a zoom-in of the \ce{^13CO2}$Q$ branch on the same vertical scale as {the top panel}, where the emission from \ce{^12CO2} and \ce{H2O} has been subtracted. The best-fit slab model is plotted in the purple shaded region. Bottom: a further zoom-in of the middle panel {where the best-fit model (180 K;} purple shaded region) {and slab models of} 300 K (blue line) and 500 K (red line) are shown. The latter two models have their emitting radius scaled to {produce the same peak flux}. In green, a slab model {with} the derived \ce{^12CO2} parameters (see text) is shown. (*fig:slab_13CO2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.12715"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arcs}{\arcsec\xspace}$
$\newcommand{\CII}{[\ion{C}{II}]158\mum}$
$\newcommand{\Ha}{\mathrm{H}\alpha\xspace}$
$\newcommand{\Hb}{\mathrm{H}\beta\xspace}$
$\newcommand{\Hg}{\mathrm{H}\gamma\xspace}$
$\newcommand{\Hd}{\mathrm{H}\delta\xspace}$
$\newcommand{\OIIlines}{[\mathrm{O}\textsc{ii}] \lambda\lambda 3727,3730\xspace}$
$\newcommand{\NeIII}{[\mathrm{Ne}\textsc{iii}] \lambda 3869\xspace}$
$\newcommand{\OIIIb}{[\mathrm{O}\textsc{iii}] \lambda 5008\xspace}$
$\newcommand{\OIIITe}{[\mathrm{O}\textsc{iii}] \lambda 4364\xspace}$
$\newcommand{\oddpm}[2]{\raisebox{0.5ex}{\tiny\substack{+#1 \ -#2}}}$</div>



<div id="title">

# Insight into the Starburst Nature of Galaxy GN-z11 with JWST MIRI Spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.12826-b31b1b.svg)](https://arxiv.org/abs/2412.12826)<mark>Appeared on: 2024-12-18</mark> -  _accepted for publication in A&A (17 pages, 6 figures, 2 tables)_

</div>
<div id="authors">

J. Álvarez-Márquez, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** This paper presents a deep MIRI/JWST medium resolution spectroscopy (MRS) covering the rest-frame optical spectrum of the GN-z11 galaxy. The [ O III ] 5008 $Å$ and H $\alpha$ emission lines are detected and spectroscopically resolved. The line profiles are well-modeled by a narrow Gaussian component with intrinsic FWHMs of 189 $\pm$ 25 and 231 $\pm$ 52 km s $^{-1}$ , respectively. We do not find any evidence of a dominant broad H $\alpha$ emission line component tracing a Broad Line Region in a type 1 active galactic nuclei (AGN). The existence of an accreting black hole dominating the optical continuum and emission lines of GN-z11 is  not compatible with the measured H $\alpha$ and [ O III ] 5008 $Å$ luminosities. If the well established relations for low- $z$ AGNs apply in GN-z11, the [ O III ] 5008 $Å$ and H $\alpha$ luminosities would imply extremely large Super-Eddington ratios ( $\lambda_{\mathrm{E}}$ $>$ 290), and bolometric luminosities $\sim$ 20 times those derived from the UV/optical continuum. However, a broad ( $\sim$ 430 $-$ 470 km s $^{-1}$ ) and weak ( $<$ 20-30 \% ) H $\alpha$ line component, tracing a minor AGN contribution in the optical, cannot be ruled out completely with the sensitivity of the present data. The physical and excitation properties of the ionized gas are consistent with a low-metallicity starburst forming stars at a rate of SFR(H $\alpha$ ) $=$ 24 $\pm$ 3 $M_{\odot}$ yr $^{-1}$ . The electron temperature of the ionized gas is $T_{\mathrm{e}}$ (O $^{++}$ ) $=$ 14000 $\pm$ 2100 K, while the direct- $T_{\mathrm{e}}$ gas-phase metallicity is 12 $+$ $\log$ (O/H) $=$ 7.91 $\pm$ 0.07 (Z = 0.17 $\pm$ 0.03 Z $_{\odot}$ ). The optical line ratios locate GN-z11 in the starburst or AGN region but more consistent with those of local low-metallicity starbursts and high- $z$ luminous galaxies detected at redshifts similar to GN-z11. We conclude that the MRS optical spectrum of GN-z11 is consistent with that of a massive, compact, and low-metallicity starburst galaxy. Due to its high SFR and stellar mass surface densities, close to that of the densest stellar clusters, we speculate that GN-z11 could be undergoing a feedback-free, highly efficient starburst phase. Additional JWST data are needed to validate this scenario, and other recently proposed alternatives, to explain the existence of bright compact galaxies in the early Universe.

</div>

<div id="div_fig1">

<img src="tmp_2412.12826/./Emission_line_plot_newcal.png" alt="Fig2" width="100%"/>

**Figure 2. -** View of the rest-frame optical spectrum of GN-z11 by zooming in the H$\beta$, [O III] 4960,5008$Å$, and H$\alpha$ emission lines. Back continuous line: 1D extracted MRS spectrum. Gray area: $\pm1\sigma$ noise calculated from the standard deviation of the local background. Red area: spectral range used to calculate the integrated line flux. Black vertical dashed line: wavelength of the peak of each emission line considering a redshift of 10.602. (*fig:Emission_Lines*)

</div>
<div id="div_fig2">

<img src="tmp_2412.12826/./Emission_line_fit_plot_1comp_newcal.png" alt="Fig3" width="100%"/>

**Figure 3. -** MRS [O III] 5008$Å$ and H$\alpha$ emission line fits. Left and right panels shows the one-component Gaussian fits, together with the fit residuals, for the [O III] 5008$Å$ and H$\alpha$ emission lines, respectively. Back continuous line: 1D extracted MRS spectrum. Gray area: $\pm1\sigma$ uncertainty calculated from the standard deviation of the local background. Green dashed line: one-components Gaussian function that best fits the spectra. Black vertical dashed line: wavelength at the peak of each emission line considering a redshift of 10.602. The $\chi_{\nu}^{2}$ for each emission line fit calculated in the velocity range, $-$500 < $v$[km s$^{-1}$] < 500, is included. (*fig:Emission_Lines_fit*)

</div>
<div id="div_fig3">

<img src="tmp_2412.12826/./M_R_newcal.png" alt="Fig6" width="100%"/>

**Figure 6. -** Mass-radius relation for GN-z11 including young star clusters in nearby starbursts (NGC253,  \citealt{Leroy2018}; M82, \citealt{McCrady2003, McCrady-Graham2007}), low-$z$ blue compact (ESO338-IG04, \citealt{Ostlin2007}) and low-metallicity (SBS0335-052E, \citealt{Adamo2010}) galaxies.
      Also represented are the values for high-$z$ clusters (Sunburst, \citealt{Vanzella-Sunburst2022}; Sunrise, \citealt{Vanzella+23}; SPT0615-JD1, \citealt{Adamo+24}), clumps (SMACS0723, \citealt{Claeyssens+Adamo2023}), extremely UV-bright SFG (J1316+2614, \citealt{Marques-Chaves+24_SFE}), and luminous galaxies at redshifts above 8 (GN-z8-LAE, \citealt{Navarro-Carrera+24}; CEERS-1019, \citealt{Marques-Chaves2024}; MACS1149-JD1, \citealt{Bradac+24}; GN-z9p4, \citep{Curti+24, Schaerer-Rui2024}; RXJ2129-z95, \citealt{Williams+23}; MACS0647-JD, \citealt{Hsiao+23-NIRCam}; GHz2, \citealt{Calabro2024}; GS-z14-0, \citealt{Helton2024}). The mass$-$size relation derived for $z$ = 4$-$10 galaxies identified with JWST \citep{Langeroodi-mass-size2023} is shown (blue line) as reference. The dotted lines represent constant stellar mass surface density in units of $M_{\odot}$ pc$^{-2}$. The line of log$_{10}\Sigma$ = 5.5 (in red) indicates the observed maximum value in clusters and nucleus of galaxies, and also predicted in dense systems under ineffective feedback regulated conditions \citep{Grudic2019}. Note that the stellar mass of GN-z11 is the average of the values presented in \citet{Bunker+23} and \citet{Tacchella+23}. (*fig:Mass-size-clusters*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.12826"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\aj}{Astronomical Journal}$
$\newcommand{\apj}{Astrophysical Journal}$
$\newcommand{\aap}{Astronomy \& Astrophysics}$
$\newcommand{\jcap}{Journal of Cosmology and Astroparticle Physics}$
$\newcommand{\aapr}{The Astronomy and Astrophysics Review}$
$\newcommand{\mnras}{Monthly Notices of the Royal Astronomical Society}$
$\newcommand{\araa}{Annual Review of Astronomy and Astrophysics}$
$\newcommand{\Msun}{\mathrm{M}_\odot}$</div>



<div id="title">

# Binary properties of the globular cluster 47 Tuc (NGC 104): A dearth of short-period binaries

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.13189-b31b1b.svg)](https://arxiv.org/abs/2412.13189)<mark>Appeared on: 2024-12-18</mark> -  _Accepted for publication in Astronomy and Astrophysics, 18 pages, 20 figures_

</div>
<div id="authors">

<mark>J. Müller-Horn</mark>, et al.

</div>
<div id="abstract">

**Abstract:** Spectroscopic observations of binary stars in globular clusters are essential to shed light on the poorly constrained period, eccentricity, and mass ratio distributions and to develop an understanding of the formation of peculiar stellar objects.   47 Tuc (NGC 104) is one of the most massive Galactic globular clusters, with a large population of blue stragglers and with many predicted but as-yet elusive stellar-mass black holes. This makes it an exciting candidate for binary searches.   We present a multi-epoch spectroscopic survey of 47 Tuc with the VLT/MUSE integral field spectrograph to determine radial velocity variations for 21,699 stars.   We find a total binary fraction in the cluster of $(2.4\pm1.0)\%$ , consistent with previous photometric estimates, and an increased binary fraction among blue straggler stars, approximately three times higher than the cluster average. We find very few binaries with periods below three days, and none with massive dark companions. A comparison with predictions from state-of-the-art models shows that the absence of such short-period binaries and of binaries with massive companions is surprising, highlighting the need to improve our understanding of stellar and dynamical evolution in binary systems.

</div>

<div id="div_fig1">

<img src="tmp_2412.13189/./figures/results/freq_vs_amp_scatter_cross_kmeans_hard.png" alt="Fig16" width="100%"/>

**Figure 16. -** \textcolor{red}{I like this plot because it shows the substantial discrepancies wrt CMC very well but am not sure whether to include it because it's a bit redundant with the period dist plot}. (*fig:period_distribution*)

</div>
<div id="div_fig2">

<img src="tmp_2412.13189/./figures/results/frequency_hist_gold.png" alt="Fig24.1" width="50%"/><img src="tmp_2412.13189/./figures/results/frequency_hist_models.png" alt="Fig24.2" width="50%"/>

**Figure 24. -** Period distribution of binaries with well-constrained orbits in 47 Tuc (shown in blue). The dotted line represents the subset of binaries with MS primaries. For comparison, the orange lines depict the period distributions of hypothetically detectable binaries from the CMC simulation (stellar types as shown in Fig. \ref{fig:ktype_heatmap}), with the dashed line indicating the subset of simulated MS-MS binaries. The red curve illustrates the predicted observable distribution assuming an underlying field-like period distribution with $\overline{\log P} \approx 5.0$ and standard deviation $\sigma_{\log P} \approx 2.3$. It is scaled to the number of well-constrained MUSE binaries. For CMC and field binaries, we have forward-modeled our selection function to account for decreasing sensitivity at longer periods. The underlying period distributions of field and CMC binaries are shown for reference in the right-hand panel. (*fig:period_distribution*)

</div>
<div id="div_fig3">

<img src="tmp_2412.13189/./figures/results/frequency_hist_gold.png" alt="Fig27" width="100%"/>

**Figure 27. -** Period distribution of binaries with well-constrained orbits in 47 Tuc (shown in blue). The dotted line represents the subset of binaries with MS primaries. For comparison, the orange lines depict the period distributions of binaries from the CMC simulation, with the dashed line indicating the subset of MS-MS binaries. The red curve illustrates the predicted observable distribution assuming an underlying field-like period distribution with $\overline{\log P} \approx 5.0$ and standard deviation $\sigma_{\log P} \approx 2.3$. It is scaled to the number of well-constrained MUSE binaries. For CMC and field binaries, we have forward-model our selection function to account for decreasing sensitivity at longer periods. (*fig:period_distribution*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.13189"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

289  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

15  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
